In [ ]:
!pip install yfinance pandas nltk scikit-learn tensorflow keras


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC, SVR
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression

# Define the list of tickers
tickers = ['MSFT', 'AAPL', 'AMZN', 'IBM']

# Initialize the list to store the data
data = []

# Fetch the historical data for each ticker and simulate sentiment analysis
for ticker in tickers:
    ticker_data = yf.download(ticker, period='1y')
    ticker_data['Returns'] = ticker_data['Close'].pct_change()
    ticker_data = ticker_data.dropna()  # Drop NaN values
    data.append(ticker_data)

# Concatenate the data
data = pd.concat(data)

# Additional features
data['MA50'] = data['Close'].rolling(window=50).mean()
data = data.dropna()

# Label returns as 0 or 1 for classification
data['Label'] = (data['Returns'] > 0).astype(int)

# Split the data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Scale the data
scaler = StandardScaler()
train_data[['Returns', 'MA50']] = scaler.fit_transform(train_data[['Returns', 'MA50']])
test_data[['Returns', 'MA50']] = scaler.transform(test_data[['Returns', 'MA50']])

# Prepare data for LSTM
X_train = train_data[['Returns', 'MA50']].values.reshape(-1, 2, 1)
X_test = test_data[['Returns', 'MA50']].values.reshape(-1, 2, 1)
y_train = train_data['Label']
y_test = test_data['Label']

# Create the LSTM model
lstm_model = Sequential()
lstm_model.add(LSTM(50, input_shape=(2, 1)))
lstm_model.add(Dense(1, activation='sigmoid'))  # Binary classification
lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Define the early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=50, min_delta=0.001)

# Train the LSTM model
lstm_model.fit(X_train, y_train, epochs=100, validation_split=0.2, callbacks=[early_stopping])

# Train the SVM model
svm_model = SVC(kernel='rbf', probability=True)
svm_model.fit(train_data[['Returns', 'MA50']], train_data['Label'])

# Make predictions using the LSTM model
lstm_predictions = (lstm_model.predict(X_test) > 0.5).astype(int).flatten()

# Make predictions using the SVM model
svm_predictions = svm_model.predict(test_data[['Returns', 'MA50']])

# Combine the predictions (using majority vote or another method)
hybrid_predictions = np.where(lstm_predictions == svm_predictions, lstm_predictions, svm_predictions)

# Evaluate the hybrid model
accuracy = accuracy_score(y_test, hybrid_predictions) * 100
print("Hybrid Model Accuracy: ", accuracy, "Percent")


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Epoch 1/100
20/20 [==============================] - 5s 70ms/step - loss: 0.6655 - accuracy: 0.7856 - val_loss: 0.6533 - val_accuracy: 0.8039
Epoch 2/100
20/20 [==============================] - 0s 5ms/step - loss: 0.6307 - accuracy: 0.8527 - val_loss: 0.6203 - val_accuracy: 0.8039
Epoch 3/100
20/20 [==============================] - 0s 6ms/step - loss: 0.5899 - accuracy: 0.8658 - val_loss: 0.5758 - val_accuracy: 0.8170
Epoch 4/100
20/20 [==============================] - 0s 6ms/step - loss: 0.5364 - accuracy: 0.8740 - val_loss: 0.5195 - val_accuracy: 0.8235
Epoch 5/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4722 - accuracy: 0.8838 - val_loss: 0.4499 - val_accuracy: 0.8431
Epoch 6/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4017 - accuracy: 0.9083 - val_loss: 0.3785 - val_accuracy: 0.8824
Epoch 7/100
20/20 [==============================] - 0s 5ms/step - loss: 0.3335 - accuracy: 0.9214 - val_loss: 0.3121 - val_accuracy: 0.9216
Epoch 8/100


In [ ]:
import yfinance as yf
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Define the ticker symbol
ticker = 'MSFT'

# Fetch the historical data
data = yf.download(ticker, period='5y')

# Preprocess the data
data['Returns'] = data['Close'].pct_change()
data.dropna(inplace=True)

# Split the data into training and testing sets
train_data, test_data = data.iloc[:-100], data.iloc[-100:]

# Scale the data
scaler = StandardScaler()
train_data[['Close', 'Returns']] = scaler.fit_transform(train_data[['Close', 'Returns']])
test_data[['Close', 'Returns']] = scaler.transform(test_data[['Close', 'Returns']])

# Create the SVM model
svm_model = SVR(kernel='rbf')
svm_model.fit(train_data[['Returns']], train_data['Close'])

# Create the LSTM model
lstm_model = Sequential()
lstm_model.add(LSTM(50, input_shape=(1, 1)))
lstm_model.add(Dense(1))
lstm_model.compile(loss='mean_squared_error', optimizer='adam')
lstm_model.fit(train_data[['Returns']].values.reshape(-1, 1, 1), train_data['Close'], epochs=100, batch_size=32, verbose=0)

# Create the Regression model
reg_model = LinearRegression()
reg_model.fit(train_data[['Returns']], train_data['Close'])

# Make predictions using the hybrid model
svm_predictions = svm_model.predict(test_data[['Returns']])
lstm_predictions = lstm_model.predict(test_data[['Returns']].values.reshape(-1, 1, 1)).flatten()
reg_predictions = reg_model.predict(test_data[['Returns']])

# Ensure all predictions are in the same shape
hybrid_predictions = (svm_predictions + lstm_predictions + reg_predictions)

# Evaluate the hybrid model
svm_mse = mean_squared_error(test_data['Close'], svm_predictions)
svm_r2 = r2_score(test_data['Close'], svm_predictions)

lstm_mse = mean_squared_error(test_data['Close'], lstm_predictions)
lstm_r2 = r2_score(test_data['Close'], lstm_predictions)

reg_mse = mean_squared_error(test_data['Close'], reg_predictions)
reg_r2 = r2_score(test_data['Close'], reg_predictions)

hybrid_mse = mean_squared_error(test_data['Close'], hybrid_predictions)
hybrid_r2 = r2_score(test_data['Close'], hybrid_predictions)

print("SVM Model:")
print("MSE:", svm_mse)
print("R-squared:", svm_r2)

print("LSTM Model:")
print("MSE:", lstm_mse)
print("R-squared:", lstm_r2)

print("Regression Model:")
print("MSE:", reg_mse)
print("R-squared:", reg_r2)

print("Hybrid Model:")
print("MSE:", hybrid_mse)
print("R-squared:", hybrid_r2)


[*********************100%%**********************]  1 of 1 completed
<ipython-input-6-a66699c7240e>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data[['Close', 'Returns']] = scaler.fit_transform(train_data[['Close', 'Returns']])
<ipython-input-6-a66699c7240e>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data[['Close', 'Returns']] = scaler.transform(test_data[['Close', 'Returns']])


4/4 [==============================] - 0s 3ms/step
SVM Model:
MSE: 6.028089053751854
R-squared: -204.49510170499738
LSTM Model:
MSE: 6.100311491939608
R-squared: -206.95713521983708
Regression Model:
MSE: 6.289291856404413
R-squared: -213.39939890732722
Hybrid Model:
MSE: 5.84612650317854
R-squared: -198.29207243599694
